# Classification of Phase Transitions using Machine Learning

This notebook explores the Ising and Potts models using PyTorch and classifies phase transitions with neural networks.

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np

## Ising Model Simulation

In [2]:
def local_energy(config, i, j):
    L = config.shape[0]
    spin = config[i, j]
    neighbors = config[(i+1)%L, j] + config[(i-1)%L, j] + \
                config[i, (j+1)%L] + config[i, (j-1)%L]
    return -spin * neighbors

def metropolis_step(config, beta):
    L = config.shape[0]
    for _ in range(L * L):
        i, j = torch.randint(0, L, (2,))
        dE = 2 * local_energy(config, i, j)
        if dE <= 0 or torch.rand(1) < torch.exp(-beta * dE):
            config[i, j] *= -1
    return config

## Potts Model Simulation

In [3]:
def metropolis_potts_step(config, beta, q):
    L = config.shape[0]
    for _ in range(L * L):
        i, j = torch.randint(0, L, (2,))
        old_state = config[i, j].item()
        new_state = torch.randint(0, q, (1,)).item()
        while new_state == old_state:
            new_state = torch.randint(0, q, (1,)).item()
        delta_E = 0
        for dx, dy in [(-1,0), (1,0), (0,-1), (0,1)]:
            ni, nj = (i + dx) % L, (j + dy) % L
            delta_E += int(config[ni, nj] == old_state) - int(config[ni, nj] == new_state)
        if delta_E <= 0 or torch.rand(1).item() < torch.exp(-beta * delta_E):
            config[i, j] = new_state
    return config

## Observables (Energy and Magnetization)

In [4]:
def compute_ising_observables(config):
    L = config.shape[0]
    E, M = 0.0, config.sum().item()
    for i in range(L):
        for j in range(L):
            S = config[i, j]
            neighbors = config[(i+1)%L, j] + config[i, (j+1)%L]
            E -= S * neighbors
    return E / (L*L), M / (L*L)

## CNN Classifier

In [5]:
class PhaseClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.fc1 = nn.Linear(64 * 8 * 8, 128)
        self.fc2 = nn.Linear(128, 2)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2)
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        return self.fc2(x)

## Visualization of Magnetization

In [6]:
def plot_magnetization(T_vals, mags):
    mags = torch.tensor(mags).reshape(len(T_vals), -1)
    avg_mag = mags.abs().mean(dim=1)
    plt.plot(T_vals, avg_mag)
    plt.xlabel("Temperature T")
    plt.ylabel("Magnetization |M|")
    plt.title("Order Parameter vs Temperature")
    plt.grid(True)
    plt.show()

## Generate Dataset (Ising model)

In [7]:
def generate_ising_dataset(L, T_vals, n_samples):
    data, labels = [], []
    for T in T_vals:
        beta = 1.0 / T
        for _ in range(n_samples):
            config = torch.randint(0, 2, (L, L)) * 2 - 1
            for _ in range(100):
                config = metropolis_step(config, beta)
            data.append(config.unsqueeze(0))
            labels.append(0 if T < 2.3 else 1)  # label: 0 for low T, 1 for high T
    return torch.stack(data).float(), torch.tensor(labels)

## Train/Test Classifier

In [8]:
def train_classifier(model, train_data, train_labels, epochs=10, lr=1e-3):
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    loss_fn = nn.CrossEntropyLoss()
    for epoch in range(epochs):
        model.train()
        logits = model(train_data)
        loss = loss_fn(logits, train_labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")
    return model

In [9]:
def test_classifier(model, test_data, test_labels):
    model.eval()
    with torch.no_grad():
        preds = model(test_data).argmax(dim=1)
        acc = (preds == test_labels).float().mean().item()
        print(f"Test Accuracy: {acc*100:.2f}%")
    return acc